In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
def get_vaccine_district_final(date):
    dates_list = [date]
    STATE_NAMES = {
      'AP': 'Andhra Pradesh',
      'AR': 'Arunachal Pradesh',
      'AS': 'Assam',
      'BR': 'Bihar',
      'CT': 'Chhattisgarh',
      'GA': 'Goa',
      'GJ': 'Gujarat',
      'HR': 'Haryana',
      'HP': 'Himachal Pradesh',
      'JH': 'Jharkhand',
      'KA': 'Karnataka',
      'KL': 'Kerala',
      'MP': 'Madhya Pradesh',
      'MH': 'Maharashtra',
      'MN': 'Manipur',
      'ML': 'Meghalaya',
      'MZ': 'Mizoram',
      'NL': 'Nagaland',
      'OR': 'Odisha',
      'PB': 'Punjab',
      'RJ': 'Rajasthan',
      'SK': 'Sikkim',
      'TN': 'Tamil Nadu',
      'TG': 'Telangana',
      'TR': 'Tripura',
      'UT': 'Uttarakhand',
      'UP': 'Uttar Pradesh',
      'WB': 'West Bengal',
      'AN': 'Andaman and Nicobar Islands',
      'CH': 'Chandigarh',
      'DN': 'Dadra and Nagar Haveli and Daman and Diu',
      'DL': 'Delhi',
      'JK': 'Jammu and Kashmir',
      'LA': 'Ladakh',
      'LD': 'Lakshadweep',
      'PY': 'Puducherry',
      'TT': 'India',
     # [UNASSIGNED_STATE_CODE]: 'Unassigned',
    }

    #Change the date here to get historical dates
    # date_=datetime.strptime('2021-12-04',"%Y-%m-%d")
    # dates_list= [date_+timedelta(days=i) for i in range(4)]
    # dates_list

    source_path="../RAWCSV"

    for Date in dates_list:
        tdate = Date
        date=tdate.strftime("%d/%m/%Y")
        if not isinstance(Date,str):
            Date=Date.strftime("%Y-%m-%d")
        print(Date)
        # /home/swiadmin/test/csv/latest/
        cowin_vaccine_data_districtwise_prev=pd.read_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_districtwise.csv",header=[0,1])
        try:
            cowin_vaccine_data_districtwise_prev = cowin_vaccine_data_districtwise_prev.drop([date],axis=1)
        except:
            pass
        cowin_vaccine_data_districtwise_master=pd.DataFrame()
        for state in STATE_NAMES.keys():
            if state=="TT":
                continue
    #         print(state)
            state_=pd.read_csv(f"{source_path}/{Date}/{state}_final.csv")
            state_.dropna(subset=["District"],axis=0,inplace=True)
            state_.reset_index(inplace=True,drop=True)
            temp_state_df=pd.DataFrame()
            temp_state_df["District"]=state_["District"]
            temp_state_df["State_Code"]=state
            temp_state_df["State"]=temp_state_df["State_Code"].map(STATE_NAMES)
            temp_state_df["District_Key"]=temp_state_df.apply(lambda rw: rw["State_Code"]+"_"+rw["District"],axis=1)
            temp_state_df["First Dose Administered"]=state_["cumulativeVaccinated1NumberForDistrict"]
            temp_state_df["Second Dose Administered"]=state_["cumulativeVaccinated2NumberForDistrict"]
            temp_state_df["Total Dose Administered"]=temp_state_df["First Dose Administered"]+temp_state_df["Second Dose Administered"]
            cowin_vaccine_data_districtwise_master=pd.concat([cowin_vaccine_data_districtwise_master,temp_state_df],axis=0)

        # if not isinstance(Date,str):


        columns=pd.MultiIndex.from_tuples(zip(["District","State_Code","State",
                    "District_Key",date,date,date],
                   [" "," "," "," ","Total Doses Administered",
                    "First Dose Administered","Second Dose Administered"
                   ]))
        cowin_vaccine_data_districtwise_master.columns=columns
        cowin_vaccine_data_districtwise_master.reset_index(inplace=True,drop=True)

        cowin_vaccine_data_districtwise_latest=pd.merge(cowin_vaccine_data_districtwise_prev,cowin_vaccine_data_districtwise_master,
                                                       left_on=[("State_Code"," "),("State"," "),("District_Key"," "),("District"," ")],
                                                       right_on=[("State_Code"," "),("State"," "),("District_Key"," "),("District"," ")],
                                                       how="left")
        cowin_vaccine_data_districtwise_latest = cowin_vaccine_data_districtwise_latest.drop_duplicates()
        cowin_vaccine_data_districtwise_latest.to_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_districtwise.csv",index=False)

